In [67]:
import numpy as np
import pandas as pd
from apyori import apriori

from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt
from sklearn import metrics
import seaborn as sns

In [68]:
# Read in data into a dataframe number of rows - 6358
file = 'soccer_player_embeddings_feature_no_labels.csv'
df = pd.read_csv('../../datasets/final_embeddings/{}'.format(file))
df.head()

,Unnamed: 0,player_positions,player_fifa_api_id,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,PC12,PC13,PC14,player_positions_general
0,0,CB,218353,0.378084,0.493866,-0.218586,-0.238711,-0.500447,0.219325,0.125196,-0.119898,-0.141964,0.142511,-0.056706,0.011622,0.019336,0.023303,DEF
1,1,LB,189615,-1.046989,-0.190204,1.120495,0.563388,-0.440816,-0.198932,0.338791,0.223150,0.041387,-0.025337,0.229595,0.055748,-0.099781,0.020387,DEF
2,2,"CB, RB",140161,0.594404,0.605691,-0.234075,-0.077747,-0.538758,0.315616,-0.124424,0.063273,-0.152510,0.248130,0.055065,0.101277,-0.090169,-0.209208,DEF
3,3,CB,17725,0.643853,0.650631,-0.236806,-0.021595,-0.580045,0.301078,0.030522,0.030255,-0.134572,0.236830,0.041646,0.433323,-0.002238,-0.071184,DEF
4,4,"CAM, CM, LM",158138,-0.530779,0.024310,1.111265,-0.766894,0.325830,-0.241209,-0.423695,0.052389,0.165501,-0.052863,0.004543,0.038298,-0.022417,-0.046864,MID


In [73]:
org_pos = df['player_positions']

In [103]:
def assoc_mining(old_pos, save_link=None):
    # Convert df to list for ARM
    old_pos_df = old_pos.apply(lambda x: sorted(x.split(", ")))
    old_pos_ls = list(old_pos_df)
    
    # Do ARM
    # Min support chosen as 0.004 because 27.6 (mean of value counts) /6358 --> ~0.0043
    association_rules = list(apriori(old_pos_ls, min_support=0.0043, min_confidence=0.5, min_length=2))
    
    arm_df = pd.DataFrame(columns=['freq_itemset', 'antecedent','consequent','support','confidence', 'lift'])
    freq_itemsets = []
    # Append results of ARM to pandas DF
    for item in association_rules:
        freq_itemsets.append(list(item[0]))
        for stats in item[2]:
            new_row = {'freq_itemset': list(item[0]),
                       'antecedent': list(stats[0]),
                       'consequent': list(stats[1]),
                       'support': item[1],
                       'confidence': stats[2],
                       'lift': stats[3]}
            arm_df = arm_df.append(new_row, ignore_index=True)
    if save_link:
        arm_df.to_csv('{}/ARM_results.csv'.format(save_link), header=True, index = False)
    
    new_pos_df = old_pos_df.copy()
    for target in freq_itemsets[:3]:
        rep_str = '_'.join(sorted(target))
        for i, row in enumerate(old_pos_df):
            for t in target:
                if t in row:
                    row.remove(t)
                    row.append(rep_str)
            new_pos_df[i] = sorted(list(set(row)))
            
    # Convert new position df to a list
    new_pos = list(new_pos_df)
    
    return new_pos, new_pos_df

In [104]:
new_pos, new_pos_df = assoc_mining(org_pos, save_link=None)